In [5]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [6]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"
numerosKalabas=[1,2,3,4,5]
#numerosKalabas=[1]
nomKalabas=[repertoire+"16-K%d/"%num for num in numerosKalabas]
print "\n".join(nomKalabas)

lexiqueTestPrep=u"dans"
lexiqueTestNoun=u"loups"
lexiqueTestHyper=u"villagEOIs"

nomDeclarationRad="Declarations-Radicaux.tex"
nomTableauxRad="Tableaux-Gloses.yaml"
nomDeclaration="Declarations.tex"
nomTableaux="Tableaux.yaml"


/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1/
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K2/
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K4/
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K5/


### Javascript from :
https://stackoverflow.com/questions/12544056/how-to-i-get-the-current-ipython-notebook-name

In [7]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [8]:
print(theNotebook)

Kalaba-Lexique-Series


In [9]:
debug=0
print_glose=True
print_cloze=True
print_radicaux=False
cloze_expanded=True
numeros={'1':'Un','2':'Deux','3':'Trois','4':'Quatre','5':'Cinq'}
personnes={'1sg':'UnSg','2sg':'DeuxSg','3sg':'TroisSg','1du':'UnDu','2du':'DeuxDu','3du':'TroisDu','1pl':'UnPl','2pl':'DeuxPl','3pl':'TroisPl'}

commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}\\grapho{%s}}"
#commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}{\\dn %s}}"
#commandGrapho=u"\\newcommand{\\%s}{\\strutgb{0pt}{%s}}"
commandPhono=u"\\newcommand{\\%sP}{\\textipa{%s}}"
commandGlose=u"\\newcommand{\\%sG}{\\textGlose{%s}}"

In [10]:
def parse_grapho(graphie):
    chunks=re.findall(ur"\s+|[ptkbdgmnNfsSvzZjwrlyv]?[aeiou]?", graphie,re.U)
    result=[]
    for chunk in chunks:
        #ATTENTION : YAML interprète "no" comme FALSE
        if chunk in syllabes.keys():
            result.append(syllabes[chunk])
        else:
            result.append(chunk)
    return "".join(result)

In [11]:
def parse_cloze(glose):
    if debug: print [glose]
    chunks=re.findall(r"\\cacherGloses{([^}]*)?}|(\w+)", glose,re.UNICODE)
    result=[]
    for chunk in chunks:
        result.extend([x for x in chunk if x!=""])
    return (u"%s;"%len(result)+";".join(result),"".join(result))

In [12]:
def recoder(chaine,table,boolUTF8=True):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    if boolUTF8:
        return result
    else:
        return result.encode("utf8")
#translit=string.maketrans(u'iueoaftgzZvjkSpN',u'tgazpHTGZJVkXyxI')

In [13]:
for serie in nomKalabas:
    print "Génération lexique :",serie.split("/")[-2], time.strftime("%H:%M:%S")


    '''
    Lecture des fichiers de paramètres
     1) Phonology.yaml pour 
         - les voyelles, les consonnes
         - la correspondance pour l'écrit (symbole => lettre, syllabe => lettres)
         - la correspondance pour l'oral
         - la désaccentuation et les déligatures du français
         - les gabarits ?
         - l'apophonie, les mutations consonantiques
     2) Stems.yaml pour
         - les catégories et les classes
         - les radicaux et les formes correspondantes
             - la première forme sert de forme de citation
     3) MorphoSyntax.yaml pour
         - définition de la valeur des formes verbales du français
         - définition des attributs de chaque catégorie
         - valeur par défaut pour chaque attribut
         - définition des cas pour SUJ, OBJ, IND et PREP
         - .../...
         - définition des constituants et de l'ordre dans les syntagmes
         - définition des contractions du français (du => de le, etc.)
    '''
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
    with open(serie+"Stems.yaml", 'r') as stream:
        stems=yaml.load(stream)
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        
    '''
    Récupération des données du ParFuMor
    '''
    with open(serie+"Hierarchie-S2.pkl", 'rb') as input:
       PFM.hierarchieCF = pickle.load(input)
    with open(serie+"Lexique-S2.pkl", 'rb') as input:
       PFM.lexique = pickle.load(input)
    with open(serie+"Regles-S2.pkl", 'rb') as input:
       PFM.regles = pickle.load(input)

    if debug: print PFM.lexique.lexemes[lexiqueTestPrep]

    consonnes=phonology["consonnes"]
    voyelles=phonology["voyelles"]
    gabarits=phonology["gabarits"]
    derives=phonology["derives"]
    nom_classe=phonology["nom_classe"]
    nom_apo=phonology["apophonies"]
    nom_mut=phonology["mutations"]
    syllabes=phonology["syllabes"]

    if "Cas" in morphosyntax:
        casSyntagmes=morphosyntax["Cas"]
    else:
        casSyntagmes=""
    lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
    casPreposition={}
    for preposition in lexiquePrepositions:
        prep=preposition.upper()
        if casSyntagmes and prep in casSyntagmes:
            casPreposition[preposition]=casSyntagmes[prep]
        elif casSyntagmes and "PREP" in casSyntagmes:
            casPreposition[preposition]=casSyntagmes["PREP"]
        else:
            casPreposition[preposition]=""
    glosePREP={x:u"%s%s"%(x.upper(),cacherGloses("["+casPreposition[x].strip("+").capitalize()+"]")) for x in casPreposition if "+" in casPreposition[x]}

    try:
        __IPYTHON__ 
        ipython=True
    except: 
        ipython=False

    
    '''
    Constitution d'un entête pour les fichiers TEX en sortie
        - nom du script
        - date du run
    '''
    texHeader=[]
    version=theNotebook
    texHeader.append("%% script : "+version)
    texHeader.append('%%%% run : %s' % time.strftime("%y%m%d-%H%M"))
    if debug: print "\n".join(texHeader)
    
    if ipython or True:
    #    lexeme_nom="lexemes.txt"
    #    phrase_nom="phrases.txt"
        pass
    else:
        parser=optparse.OptionParser()
        parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
        parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
        parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
        parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

        (options, args) = parser.parse_args()
        lexeme_nom=args[0]
        phrase_nom=args[1]

    phonoIn =  unicode(phonology["translations"]["grapho"]["in"])
    graphoIn = [ord(char) for char in phonoIn]
    graphoOut = unicode(phonology["translations"]["grapho"]["out"])
    translit = dict(zip(graphoIn, graphoOut))

    accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
    deaccentIn = [ord(char) for char in accentedIn]
    deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
    deaccent = dict(zip(deaccentIn, deaccentOut))

    deligatures=phonology["translations"]["deligatures"]

    tipaIn = unicode(phonology["translations"]["ipa"]["in"])
    ipaIn = [ord(char) for char in tipaIn]
    ipaOut = unicode(phonology["translations"]["ipa"]["out"])
    toipa = dict(zip(ipaIn, ipaOut))

    tableaux={}
    tableauxGloses={}
    gloseClozes={}
    declarations=[]
    declarationsRad=[]
    
    '''
    Pour chaque catégorie
    '''
    for categorie in PFM.lexique.catLexeme:
        '''
        si la catégorie n'est pas dans le tableau, il faut l'ajouter dans les trois structures :
            tableaux : les lignes à ajouter à la fin du document LaTeX sans les gloses
            tableauxGloses : les lignes à ajouter à la fin du document LaTeX avec les gloses
            gloseClozes : les informations sur les mots pour les questions CLOZE 
        '''
        if not categorie in tableaux:
            tableaux[categorie]=set()
            tableauxGloses[categorie]=set()
            gloseClozes[categorie]=[]
        if verbose: print categorie
        '''
        pour chaque lexème de la catégorie
        '''
        for lexeme in PFM.lexique.catLexeme[categorie]:
            if verbose: print PFM.lexique.lexemes[lexeme]
            '''
            pour chaque case du paradigme du lexème
            '''
            for case in PFM.lexique.lexemes[lexeme].paradigme.cases:
                caseGlose=case.glose
                '''
                Les éléments des catégories majeures (N,V,Adj) sont représentés en minuscules
                    => les noms propres commencent par une capitale
                Les éléments grammaticaux sont représentés en majuscules
                '''
                if categorie in PFM.categoriesMajeures:
                    nom=PFM.lexique.lexemes[lexeme].nom
                else:
                    nom=PFM.lexique.lexemes[lexeme].nom.upper()
                    gloseCase=case.glose
                    if categorie=="PREP" and gloseCase in glosePREP:
                        caseGlose=glosePREP[gloseCase]
                    elif categorie=="PREP":
                        caseGlose=caseGlose.upper()
                ref=PFM.modifierGlose(nom,case.sigma,"ref")
                ref=recoder(ref,deaccent)
                
                
                '''
                Dealing with LaTeX names format for commands
                
                 1) no ligatures
                 2) no numbers
                '''
                
                for ligature in deligatures:
                    ref=ref.replace(ligature,deligatures[ligature])
                for pers in personnes:
                    ref=ref.replace(pers,personnes[pers])
                for num in numeros:
                    ref=ref.replace(num,numeros[num])
                    
                grapho=chaine2utf8(re.sub(ur"\s+",ur"",recoder(parse_grapho(case.forme),translit)))
                phonoRad=case.detoure.replace(" ","")
                phono=case.forme.replace(" ","")

                '''
                Creating the DECLARATIONS TEX contents
                '''
                
                declarations.append(commandGrapho%(ref,grapho))
                declarations.append(commandPhono%(ref,phono))
                declarations.append(commandGlose%(ref,caseGlose))
                declarationsRad.append(commandGrapho%(ref,grapho))
                declarationsRad.append(commandPhono%(ref,phonoRad))
                declarationsRad.append(commandGlose%(ref,caseGlose))
                
                '''
                Creating the TABLEAUX TEX contents
                '''
                
                tableauxGloses[categorie].add("\\"+ref+" & \\"+ref+"P & \\"+ref+"G \\\\")
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\blanc{\\"+ref+"G} \\\\")
                
                '''
                Creating the CLOZE CSV contents
                '''
                
                vedette=nom.split(".")[0]
                gloses,strGlose=parse_cloze(case.glose)
                phono=case.forme.replace(" ","")
                if debug: print [grapho, recoder(phono,toipa)]
                if cloze_expanded:
                    try:
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),recoder(case.decoupe,toipa),
                                         case.sigma,
                                         strGlose,
                                         gloses])
                    except NameError:
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),
                                         strGlose,
                                         gloses])
                else:
                    try:
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),recoder(case.decoupe,toipa),case.sigma,gloses])
                    except NameError:
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),gloses])
                gloseClozes[categorie].append(cloze)

        '''
        Écriture des fichiers en sortie
         1) déclarations sans les radicaux
         2) déclarations avec les radicaux
         3) données pour les questions CLOZE
         4) tableaux de vocabulaire avec gloses
         5) tableaux de vocabulaire sans gloses
        '''
        with codecs.open(serie+nomDeclaration, 'wb', encoding='utf8') as output:
            for declaration in texHeader+declarations:
        #        print type(declaration),declaration
                output.write(declaration+"\n")

        with codecs.open(serie+nomDeclarationRad, 'wb', encoding='utf8') as output:
            for declaration in texHeader+declarationsRad:
        #        print type(declaration),declaration
                output.write(declaration+"\n")

        with codecs.open(serie+"Clozes.txt", 'wb', encoding='utf8') as output:
            for categorie in gloseClozes:
                output.write("#\t"+categorie+"\n#\n#\n")
                for cloze in gloseClozes[categorie]:
                    output.write(cloze+"\n")
                output.write("#\n#\n#\n")

        yaml.safe_dump(tableauxGloses, file(serie+nomTableauxRad, 'w'), encoding='utf-8', allow_unicode=True)
        yaml.safe_dump(tableaux, file(serie+nomTableaux, 'w'), encoding='utf-8', allow_unicode=True)

print u"Génération terminée :", time.strftime("%H:%M:%S")

Génération lexique : 16-K1 16:38:52
Génération lexique : 16-K2 16:38:53
Génération lexique : 16-K3 16:38:55
Génération lexique : 16-K4 16:38:59
Génération lexique : 16-K5 16:39:03
Génération terminée : 16:39:06


In [14]:
#gloseClozes

In [15]:
[l for l in declarations if "enfant" in l]

[u'\\newcommand{\\enfantASgErg}{\\strutgb{0pt}\\grapho{keqamovaa}}',
 u'\\newcommand{\\enfantASgErgP}{\\textipa{kezamovaa}}',
 u'\\newcommand{\\enfantASgErgG}{\\textGlose{\\cacherGloses{N12.Erg-}enfant\\cacherGloses{.N1.N12.A}\\cacherGloses{-N1.Sg}}}',
 u'\\newcommand{\\enfantASgAbs}{\\strutgb{0pt}\\grapho{qamovaake}}',
 u'\\newcommand{\\enfantASgAbsP}{\\textipa{zamovaake}}',
 u'\\newcommand{\\enfantASgAbsG}{\\textGlose{enfant\\cacherGloses{.N1.N12.A}\\cacherGloses{-N1.Sg}\\cacherGloses{-N12.Abs}}}',
 u'\\newcommand{\\enfantASgDat}{\\strutgb{0pt}\\grapho{qamovaaso}}',
 u'\\newcommand{\\enfantASgDatP}{\\textipa{zamovaaso}}',
 u'\\newcommand{\\enfantASgDatG}{\\textGlose{enfant\\cacherGloses{.N1.N12.A}\\cacherGloses{-N1.Sg}\\cacherGloses{-N12.Dat}}}',
 u'\\newcommand{\\enfantASgObl}{\\strutgb{0pt}\\grapho{qamovaamu}}',
 u'\\newcommand{\\enfantASgOblP}{\\textipa{zamovaamu}}',
 u'\\newcommand{\\enfantASgOblG}{\\textGlose{enfant\\cacherGloses{.N1.N12.A}\\cacherGloses{-N1.Sg}\\cacherGloses{-N